In [1]:
import pandas as pd
import numpy as np
import os 
import sys
import glob 
import json
import importlib
import tqdm
import warnings
import matplotlib.pyplot as plt
import rapidfuzz as rf
import pickle

In [2]:
#Make sure to run this only once, to make lobbying-market the working directory
parent_dir_name=os.path.basename(os.path.dirname(os.getcwd()))
if parent_dir_name=='lobbying-market':
    os.chdir(os.path.pardir)
    print(os.getcwd())
    utils_path= os.path.abspath('utils')
    sys.path.append(utils_path)

import preprocessing as pp
importlib.reload(pp)


c:\Users\nicoj\OneDrive\Documents\GitHub\lobbying-market


<module 'preprocessing' from 'c:\\Users\\nicoj\\OneDrive\\Documents\\GitHub\\lobbying-market\\utils\\preprocessing.py'>

In [21]:
lobby_by_company= pd.read_csv('LDA_data/data_by_company.csv', index_col='ticker')

In [ ]:
# lobby_by_company.to_csv('LDA_data/data_by_company.csv')

In [4]:
lobby_by_company.shape

(1408, 221)

In [8]:
alist=['1','2','3','4']
qtkey_list= []
for i in range(2013,2024):
    qtkey_list=qtkey_list + [str(i)+key for key in alist]

propkey_list=['count','nan','income','expenses','total']
prop_dict={prop_key: [prop_key+'_'+ qt_key for qt_key in qtkey_list] for prop_key in propkey_list}
print(prop_dict['count'])

['count_20131', 'count_20132', 'count_20133', 'count_20134', 'count_20141', 'count_20142', 'count_20143', 'count_20144', 'count_20151', 'count_20152', 'count_20153', 'count_20154', 'count_20161', 'count_20162', 'count_20163', 'count_20164', 'count_20171', 'count_20172', 'count_20173', 'count_20174', 'count_20181', 'count_20182', 'count_20183', 'count_20184', 'count_20191', 'count_20192', 'count_20193', 'count_20194', 'count_20201', 'count_20202', 'count_20203', 'count_20204', 'count_20211', 'count_20212', 'count_20213', 'count_20214', 'count_20221', 'count_20222', 'count_20223', 'count_20224', 'count_20231', 'count_20232', 'count_20233', 'count_20234']


In [7]:
len([num for num in lobby_by_company['count_final'].values if num >0])

1408

In [9]:
qtkey_df=pd.DataFrame()
qtkey_df['qt_key']=qtkey_list
print(qtkey_df.head())
qtkey_df.set_index('qt_key',inplace=True)

  qt_key
0  20131
1  20132
2  20133
3  20134
4  20141


In [29]:
def make_df_from_ticker (ticker_key):
    result= qtkey_df
    for prop_key in prop_dict:
        col_list= prop_dict[prop_key]
        value_list=lobby_by_company[prop_dict[prop_key]].loc[ticker_key].values
        result[prop_key]=value_list
    return result

In [28]:
make_df_from_ticker('MSFT').head()

,count,nan,income,expenses,total
qt_key,,,,,
20131,23.0,0.0,1057500.0,2530000.0,3587500.0
20132,23.0,0.0,980000.0,2960000.0,3940000.0
20133,23.0,0.0,987000.0,2230000.0,3217000.0
20134,24.0,0.0,1070000.0,2770000.0,3840000.0
20141,22.0,0.0,937000.0,2080000.0,3017000.0


In [19]:
lobby_by_company.head()

,count_20131,nan_20131,income_20131,expenses_20131,total_20131,count_20132,nan_20132,income_20132,expenses_20132,total_20132,...,nan_20233,income_20233,expenses_20233,total_20233,count_20234,nan_20234,income_20234,expenses_20234,total_20234,count_final
ticker,,,,,,,,,,,,,,,,,,,,,
MSFT,23.0,0.0,1057500.0,2530000.0,3587500.0,23.0,0.0,980000.0,2960000.0,3940000.0,...,0.0,885000.0,2330000.0,3215000.0,20.0,0.0,955000.0,2120000.0,3075000.0,957.0
AAPL,6.0,0.0,260000.0,720000.0,980000.0,6.0,0.0,260000.0,690000.0,950000.0,...,1.0,800000.0,1840000.0,2640000.0,11.0,1.0,800000.0,2950000.0,3750000.0,340.0
NVDA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,160000.0,0.0,160000.0,2.0,0.0,160000.0,0.0,160000.0,10.0
AMZN,3.0,0.0,260000.0,856831.0,1116831.0,5.0,0.0,250000.0,860000.0,1110000.0,...,0.0,790000.0,4340000.0,5130000.0,16.0,0.0,770000.0,4460000.0,5230000.0,419.0
GOOGL,21.0,0.0,1095000.0,3350000.0,4445000.0,21.0,0.0,1065000.0,3360000.0,4425000.0,...,1.0,975000.0,2960000.0,3935000.0,20.0,1.0,950000.0,2820000.0,3770000.0,903.0


In [23]:
company_df_dict={key: make_df_from_ticker(key) for key in lobby_by_company.index}

'CASA'

In [46]:
company_df_list=[ make_df_from_ticker(key) for key in list(lobby_by_company.index) ]

In [47]:
company_df_list[9].head()

,count,nan,income,expenses,total
qt_key,,,,,
20131,0.0,0.0,0.0,0.0,0.0
20132,0.0,0.0,0.0,0.0,0.0
20133,0.0,0.0,0.0,0.0,0.0
20134,0.0,0.0,0.0,0.0,0.0
20141,0.0,0.0,0.0,0.0,0.0


In [12]:
ticker_df=lobby_by_company[[]]

In [14]:
company_data_in_df=ticker_df

In [43]:
company_df_dict['data_f']=company_df_list

ValueError: Length of values (0) does not match length of index (1408)

In [44]:
len(company_df_list)

0

In [47]:
company_df_dict['CASA'].head()

KeyError: 'CASA'

In [20]:
company_df_dict['MSFT'].head()

,count,nan,income,expenses,total
qt_key,,,,,
20131,0.0,0.0,0.0,0.0,0.0
20132,0.0,0.0,0.0,0.0,0.0
20133,0.0,0.0,0.0,0.0,0.0
20134,0.0,0.0,0.0,0.0,0.0
20141,0.0,0.0,0.0,0.0,0.0
